# Agent Backed by Azure OpenAI Assistant API

The GPTAssistantAgent is a powerful component of the AutoGen framework, utilizing OpenAI's Assistant API to enhance agents with advanced capabilities. This agent enables the integration of multiple tools such as the Code Interpreter, File Search, and Function Calling, allowing for a highly customizable and dynamic interaction model.

Version Requirements:

- AutoGen: Version 0.2.27 or higher.
- OpenAI: Version 1.21 or higher.

Key Features of the GPTAssistantAgent:

- Multi-Tool Mastery: Agents can leverage a combination of OpenAI's built-in tools, like Code Interpreter and File Search, alongside custom tools you create or integrate via Function Calling.

- Streamlined Conversation Management: Benefit from persistent threads that automatically store message history and adjust based on the model's context length. This simplifies development by allowing you to focus on adding new messages rather than managing conversation flow.

- File Access and Integration: Enable agents to access and utilize files in various formats. Files can be incorporated during agent creation or throughout conversations via threads. Additionally, agents can generate files (e.g., images, spreadsheets) and cite referenced files within their responses.

### Create a OpenAI Assistant in Autogen

In [32]:
import os

from autogen.agentchat.contrib.gpt_assistant_agent import GPTAssistantAgent
from autogen import UserProxyAgent
from dotenv import load_dotenv

load_dotenv()


user_proxy = UserProxyAgent(
    "user_proxy", 
    max_consecutive_auto_reply=0,
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    }
)

assistant_id = os.environ.get("ASSISTANT_ID", None)
llm_config = {
    "config_list": [
    {
        "model": os.environ.get("OPENAI_GPT_DEPLOYMENT", None),
        "api_key": os.environ.get("OPENAI_KEY", None),
        "base_url": os.environ.get("OPENAI_URI", None),
        "api_type": "azure",
        "api_version": os.environ.get("OPENAI_VERSION", None),
    }
],
}
assistant_config = {
    # define the openai assistant behavior as you need
}
oai_agent = GPTAssistantAgent(
    name="oai_agent",
    instructions="I'm an openai assistant running in autogen",
    llm_config=llm_config,
    assistant_config=assistant_config,
)

Send a test question:

In [33]:
user_proxy.initiate_chat(recipient=oai_agent,
    message="Tell me a joke."
)

user_proxy (to oai_agent):

Tell me a joke.

--------------------------------------------------------------------------------
oai_agent (to user_proxy):

Sure, how about this one:

Why don't skeletons fight each other?

They don't have the guts.


--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'Tell me a joke.', 'role': 'assistant'}, {'content': "Sure, how about this one:\n\nWhy don't skeletons fight each other?\n\nThey don't have the guts.\n", 'role': 'user'}], summary="Sure, how about this one:\n\nWhy don't skeletons fight each other?\n\nThey don't have the guts.\n", cost={'usage_including_cached_inference': {'total_cost': 0}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=[])


### Use OpenAI Assistant Built-in Tools and Function Calling

##### Code Interpreter

The Code Interpreter empowers your agents to write and execute Python code in a secure environment provide by OpenAI. This unlocks several capabilities, including but not limited to:

- Process data: Handle various data formats and manipulate data on the fly.
- Generate outputs: Create new data files or even visualizations like graphs.

Using the Code Interpreter with the following configuration.


In [34]:
assistant_config = {
    "tools": [
        {"type": "code_interpreter"}
    ]
}

code_oai_agent = GPTAssistantAgent(
    name="code_oai_agent",
    instructions="You are a code assistant tool that can run Python code",
    llm_config=llm_config,
    assistant_config=assistant_config,
)

In [35]:
user_proxy.initiate_chat(recipient=code_oai_agent,
    message="Run a Monty Hall problem simulation 10000 times, and tell me how often you win by switching doors."
)

user_proxy (to code_oai_agent):

Run a Monty Hall problem simulation 10000 times, and tell me how often you win by switching doors.

--------------------------------------------------------------------------------
code_oai_agent (to user_proxy):

After running the Monty Hall problem simulation 10,000 times, you win approximately 6,610 times by switching doors, which corresponds to a winning probability of about 66.10%. This is consistent with the probability theory behind the Monty Hall problem, which states that by switching doors, the contestant has a 2/3 (or approximately 66.67%) chance of winning.


--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'Run a Monty Hall problem simulation 10000 times, and tell me how often you win by switching doors.', 'role': 'assistant'}, {'content': 'After running the Monty Hall problem simulation 10,000 times, you win approximately 6,610 times by switching doors, which corresponds to a winning probability of about 66.10%. This is consistent with the probability theory behind the Monty Hall problem, which states that by switching doors, the contestant has a 2/3 (or approximately 66.67%) chance of winning.\n', 'role': 'user'}], summary='After running the Monty Hall problem simulation 10,000 times, you win approximately 6,610 times by switching doors, which corresponds to a winning probability of about 66.10%. This is consistent with the probability theory behind the Monty Hall problem, which states that by switching doors, the contestant has a 2/3 (or approximately 66.67%) chance of winning.\n', cost={'usage_including_cached_inference': {'total_cos

#### Using files

To upload files, you can employ two methods:

Azure OpenAI Playground: Leverage the Azure OpenAI Playground, an interactive platform accessible at https://oai.azure.com, to upload your files and obtain the corresponding file IDs.

Code-Based Uploading: Alternatively, you can upload files and retrieve their file IDs programmatically using the following code snippet.


In [36]:
# Get sample files

!curl -L "https://github.com/Azure-Samples/cognitive-services-sample-data-files/raw/master/openai/contoso_benefits_document_example.pdf" > data/contoso_benefits_document_example.pdf
!curl -L "https://go.microsoft.com/fwlink/?LinkID=521962" > data/financial_sample.xlsx

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 65610  100 65610    0     0  77083      0 --:--:-- --:--:-- --:--:-- 77083
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 83418  100 83418    0     0   281k      0 --:--:-- --:--:-- --:--:-- 3879k


In [37]:
from openai import AzureOpenAI

client = AzureOpenAI(
    base_url=os.environ.get("BASE_URL"),
    api_key=os.environ.get("OPENAI_KEY"),
    api_version=os.environ.get("OPENAI_VERSION")
)

file = client.files.create(
  file=open("data/financial_sample.xlsx", "rb"),
  purpose='assistants'
)


In [38]:
assistant_config = {
    "tools": [
        {"type": "code_interpreter"}
    ],
    "tool_resources": {
        "code_interpreter": {
            "file_ids": [file.id]
        }
    }
}

excel_oai_agent = GPTAssistantAgent(
    name="excel_oai_agent",
    instructions="You are a code assistant tool that can run Python code",
    llm_config=llm_config,
    assistant_config=assistant_config,
)

user_proxy.initiate_chat(recipient=excel_oai_agent,
    message="What are the columns in the excel spreadsheet uploaded?"
)

user_proxy (to excel_oai_agent):

What are the columns in the excel spreadsheet uploaded?

--------------------------------------------------------------------------------
excel_oai_agent (to user_proxy):

The columns in the Excel spreadsheet are:

1. Segment
2. Country
3. Product
4. Discount Band
5. Units Sold
6. Manufacturing Price
7. Sale Price
8. Gross Sales
9. Discounts
10. Sales
11. COGS
12. Profit
13. Date
14. Month Number
15. Month Name
16. Year


--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'What are the columns in the excel spreadsheet uploaded?', 'role': 'assistant'}, {'content': 'The columns in the Excel spreadsheet are:\n\n1. Segment\n2. Country\n3. Product\n4. Discount Band\n5. Units Sold\n6. Manufacturing Price\n7. Sale Price\n8. Gross Sales\n9. Discounts\n10. Sales\n11. COGS\n12. Profit\n13. Date\n14. Month Number\n15. Month Name\n16. Year\n', 'role': 'user'}], summary='The columns in the Excel spreadsheet are:\n\n1. Segment\n2. Country\n3. Product\n4. Discount Band\n5. Units Sold\n6. Manufacturing Price\n7. Sale Price\n8. Gross Sales\n9. Discounts\n10. Sales\n11. COGS\n12. Profit\n13. Date\n14. Month Number\n15. Month Name\n16. Year\n', cost={'usage_including_cached_inference': {'total_cost': 0}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=[])


### File Search

The File Search tool empowers your agents to tap into knowledge beyond its pre-trained model. This allows you to incorporate your own documents and data, such as product information or code files, into your agent's capabilities.

Here's how to obtain the vector_store.id using two methods:

OpenAI Playground: Leverage the Azure OpenAI Playground, an interactive platform accessible at https://oai.azure.com, to create a vector store, upload your files, and add it into your vector store. Once complete, you'll be able to retrieve the associated vector_store.id.

Code-Based Uploading:Alternatively, you can upload files and retrieve their file IDs programmatically using the following code snippet:


In [39]:
from openai import AzureOpenAI

client = AzureOpenAI(
    base_url=os.environ.get("BASE_URL"),
    api_key=os.environ.get("OPENAI_KEY"),
    api_version=os.environ.get("OPENAI_VERSION")
)

# Step 1: Create a Vector Store
vector_store = client.beta.vector_stores.create(name="Financial Statements")
print("Vector Store created:", vector_store.id)  # This is your vector_store.id

# Step 2: Prepare Files for Upload
file_paths = ["./data/contoso_benefits_document_example.pdf"]
file_streams = [open(path, "rb") for path in file_paths]

# Step 3: Upload Files and Add to Vector Store (with status polling)
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)

# Step 4: Verify Completion (Optional)
print("File batch status:", file_batch.status)
print("Uploaded file count:", file_batch.file_counts.completed)


Vector Store created: vs_FFMAaffcy5bgIGagoAeoXMtQ
File batch status: completed
Uploaded file count: 1


In [41]:
assistant_config = {
    "tools": [
        {"type": "retrieval"}
    ],
    "tool_resources": {
        "file_search": {
            "vector_store_ids": [vector_store.id]
        }
    }
}

files_oai_agent = GPTAssistantAgent(
    name="files_oai_agent",
    instructions="You are a files search assistant tool",
    llm_config=llm_config,
    assistant_config=assistant_config
)

user_proxy.initiate_chat(recipient=files_oai_agent,
    message="Does Contoso benefits cover dental care?"
)

user_proxy (to files_oai_agent):

Does Contoso benefits cover dental care?

--------------------------------------------------------------------------------
files_oai_agent (to user_proxy):

Yes, Contoso benefits do cover dental care. According to the Contoso electronics plan and benefit packages, there are two plans available that provide dental care:

1. **Northwind Health Plus**: This comprehensive plan offers coverage for dental services which includes dental exams, cleanings, and fillings.
2. **Northwind Standard**: This basic plan provides coverage for medical, vision, and dental services, although it only specifies coverage for vision exams and glasses, it does not explicitly state the extent of dental services covered.

Therefore, if you are considering dental coverage, the Northwind Health Plus might be the more suitable option for comprehensive dental care【0†contoso_benefits_document_example.pdf】.


-----------------------------------------------------------------------------

ChatResult(chat_id=None, chat_history=[{'content': 'Does Contoso benefits cover dental care?', 'role': 'assistant'}, {'content': 'Yes, Contoso benefits do cover dental care. According to the Contoso electronics plan and benefit packages, there are two plans available that provide dental care:\n\n1. **Northwind Health Plus**: This comprehensive plan offers coverage for dental services which includes dental exams, cleanings, and fillings.\n2. **Northwind Standard**: This basic plan provides coverage for medical, vision, and dental services, although it only specifies coverage for vision exams and glasses, it does not explicitly state the extent of dental services covered.\n\nTherefore, if you are considering dental coverage, the Northwind Health Plus might be the more suitable option for comprehensive dental care【0†contoso_benefits_document_example.pdf】.\n', 'role': 'user'}], summary='Yes, Contoso benefits do cover dental care. According to the Contoso electronics plan and benefit packag

### Function calling

Function Calling empowers you to extend the capabilities of your agents with your pre-defined functionalities, which allows you to describe custom functions to the Assistant, enabling intelligent function selection and argument generation.

Using the Function calling with the following configuration.

In [43]:
from autogen.function_utils import get_function_schema

@user_proxy.register_for_execution()
def get_current_weather(location: str) -> dict:
    """
    Retrieves the current weather for a specified location.

    Args:
    location (str): The location to get the weather for.

    Returns:
    Union[str, dict]: A dictionary with weather details..
    """

    # Simulated response
    return {
        "location": location,
        "temperature": 22.5,
        "description": "Partly cloudy"
    }

api_schema = get_function_schema(
    get_current_weather,
    name=get_current_weather.__name__,
    description="Returns the current weather data for a specified location."
)

assistant_config = {
    "tools": [
        api_schema
    ],
}

functions_oai_agent = GPTAssistantAgent(
    name="functions_oai_agent",
    instructions="You are a weather assistant",
    llm_config=llm_config,
    assistant_config=assistant_config,
    function_map={
        "get_current_weather": get_current_weather
    }
)

user_proxy.initiate_chat(recipient=functions_oai_agent,
    message="What's the weather like in Redmond, WA?"
)

user_proxy (to functions_oai_agent):

What's the weather like in Redmond, WA?

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_current_weather...


INFO:autogen.agentchat.contrib.gpt_assistant_agent:Intermediate executing(get_current_weather, Success: True) : {'location': 'Redmond, WA', 'temperature': 22.5, 'description': 'Partly cloudy'}


functions_oai_agent (to user_proxy):

The current weather in Redmond, WA is partly cloudy with a temperature of 22.5°C.


--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': "What's the weather like in Redmond, WA?", 'role': 'assistant'}, {'content': 'The current weather in Redmond, WA is partly cloudy with a temperature of 22.5°C.\n', 'role': 'user'}], summary='The current weather in Redmond, WA is partly cloudy with a temperature of 22.5°C.\n', cost={'usage_including_cached_inference': {'total_cost': 0}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=[])